In [208]:
from __future__ import print_function

import os
import sys
import glob
import h5py
import numpy as np
import math


import torch
from torch import nn
from torch.autograd import Variable
from torchvision import transforms
from torchvision.utils import save_image
from torch.utils.data import Dataset , DataLoader
import torch.optim as optim
import torch.nn.functional as F
from tqdm import tqdm
import matplotlib.pyplot as plt

import tensorboard
import tensorboardX
from torch.utils.tensorboard import SummaryWriter


from log import Logger
from data import  trainDataset, testDataset, trainlabelDataset,testlabelDataset
from util import r2, mse, rmse, mae, pp_mse, pp_rmse, pp_mae
from model import autoencoder_999, autoencoder_333

def to_img(x):   # image size 
    x = x.view(x.size(0), 1, 64, 64)
    return x




if not os.path.exists('./gal_img1001'):
    os.mkdir('./gal_img1001')

    
dataset= trainlabelDataset()
dataloader= DataLoader(dataset=dataset, batch_size=64,shuffle=True,drop_last=True)

test_dataset = testlabelDataset()
test_dataloader= DataLoader(dataset=test_dataset, batch_size=64,shuffle=True,drop_last=True)


writer = SummaryWriter("run1001/exp411",)  ################################################### change name 

num_epochs =20000
batch_size = 64
learning_rate = 5e-1

model = autoencoder_333().cuda()   ############################################################## AE model 
model.load_state_dict(torch.load('gal_img1001/exp401_7700.pth'))    ###

criterion_mean = nn.L1Loss(reduction='mean')
criterion_none = nn.L1Loss(reduction='none')



#scheduler 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[1000,4000], gamma=0.1)

In [207]:
for epoch in range(num_epochs):
    total_loss = 0.0
    total_mse = 0.0 
    total_recon=0.0
    total_latent=0.0
    
    num_examples = 0.0
    test_num_examples=0.0
    
    
    test_total_loss = 0.0    
    test_total_mse=0.0
    test_total_recon=0.0
    test_total_latent=0.0
    model.train()
    for data in dataloader:
        img,label= [x.type(torch.float32).cuda() for x in data]
        img = img.view(img.size(0), 1,64,64)

       # print(img.shape)
       # print("",img.sum())
       # print("",img[0].sum())
        # forward
        output, z = model(img)
        z=z.view(z.size(0),14*14)
       # print("output ",output.shape)
       # print("z ",z.shape)
    ################################################## Loss function with regularizing Z ########################
        
        loss= (( criterion_none(output, img)/(img.sum(dim=3).sum(dim=2).sum(dim=1))).mean())* batch_size  + criterion_mean(z[:,:7], label) /(1e10) 

        loss_recon= criterion_mean(output, img)
        loss_latent= criterion_mean(z[:,:7], label)
        
        
        MSE_loss = nn.MSELoss()(output, img)
        batch_size = img.size(0)
        total_loss += loss.item() * batch_size
        total_mse += MSE_loss.item() * batch_size
        total_recon+= loss_recon.item() * batch_size
        total_latent+= loss_latent.item() * batch_size

        num_examples += batch_size

        
        optimizer.zero_grad()
    # backward
        loss.backward()
        optimizer.step()
        scheduler.step()
        
    model.eval()
    for data in test_dataloader:
        test_img,test_label= [x.type(torch.float32).cuda() for x in data]

        test_img = test_img.view(test_img.size(0), 1,64,64)
       # print(img.shape)
        

        # forward
        test_output,test_z = model(test_img)
        test_z=test_z.view(test_z.size(0),14*14)
                
       #print("output ",output.shape)
       # test_loss = criterion(test_output, test_img) + criterion(z[:,:7], label) /(1e)  #  + 1e-5*  criterion(test_z[:,:7], test_label) 
        test_loss= (( criterion_none(test_output, test_img)/(test_img.sum(dim=3).sum(dim=2).sum(dim=1))).mean())* batch_size  + criterion_mean(test_z[:,:7], test_label) /(1e10) 
        
        test_loss_recon= criterion_mean(test_output, test_img)
        test_loss_latent= criterion_mean(test_z[:,:7], test_label)
        
        test_MSE_loss = nn.MSELoss()(test_output, test_img)
        batch_size = test_img.size(0)
        test_total_loss += test_loss.item() * batch_size
        test_total_mse += test_MSE_loss.item() * batch_size
        test_total_recon+= test_loss_recon.item() * batch_size
        test_total_latent+= test_loss_latent.item() * batch_size


        test_num_examples += batch_size

    writer.add_scalar('Loss/train',total_loss / num_examples,epoch)
    writer.add_scalar('Mse/train', total_mse / num_examples,epoch)   
    writer.add_scalar('Recon/train', total_recon / num_examples,epoch)        
    writer.add_scalar('Latent/train', total_latent / num_examples,epoch)        
    writer.add_scalar('Loss/test',test_total_loss / test_num_examples,epoch)
    writer.add_scalar('Mse/test', test_total_mse / test_num_examples,epoch)
    writer.add_scalar('Recon/test', test_total_recon / num_examples,epoch)        
    writer.add_scalar('Latent/test', test_total_latent / num_examples,epoch)   
    
    
    print('epoch [{}/{}], loss:{:.6f}, MSE_loss:{:.6f}, recon_loss:{:.6f}, latent_loss:{:.6f}'
          .format(epoch + 1, num_epochs, total_loss / num_examples, total_mse/ num_examples, total_recon / num_examples, total_latent/ num_examples))     
    
    print('epoch [{}/{}], test_loss:{:.6f}, test_MSE_loss:{:.6f}, test_recon_loss:{:.6f}, test_latent_loss:{:.6f}'
          .format(epoch + 1, num_epochs, test_total_loss / num_examples, test_total_mse/ num_examples, test_total_recon / num_examples, test_total_latent/ num_examples))   
    
    if epoch % 10 == 0:
        x = to_img(img.cpu().data)
        x_hat = to_img(output.cpu().data)
        test_x = to_img(test_img.cpu().data)    ########## change name 
        test_x_hat = to_img(test_output.cpu().data)
        torch.save(x, './gal_img1001/exp411_x_{}.pt'.format(epoch))
        torch.save(x_hat, './gal_img1001/exp411_x_hat_{}.pt'.format(epoch))
        torch.save(test_x, './gal_img1001/exp411_test_x_{}.pt'.format(epoch))
        torch.save(test_x_hat, './gal_img1001/exp411_test_x_hat_{}.pt'.format(epoch))
        torch.save(model.state_dict(), './gal_img1001/exp411_{}.pth'.format(epoch))     

epoch [1/20000], loss:0.036814, MSE_loss:0.000488, recon_loss:0.007313, latent_loss:7177893.464744
epoch [1/20000], test_loss:0.011384, test_MSE_loss:0.000150, test_recon_loss:0.002277, test_latent_loss:2144508.451923


KeyboardInterrupt: 

In [53]:
criterion(z[:,:7], label)/1e7

tensor(0.6775, device='cuda:0', grad_fn=<DivBackward0>)

In [58]:
img.shape

torch.Size([64, 1, 64, 64])

In [6]:
img[0].shape

torch.Size([1, 64, 64])

In [7]:
img[0].sum()

tensor(4.0610, device='cuda:0')

In [9]:
img[0].max()

tensor(0.0128, device='cuda:0')

In [10]:
img.sum()

tensor(1878.3997, device='cuda:0')

In [35]:
img.sum()

tensor(1878.3997, device='cuda:0')

In [ ]:
criterion(output, img)

In [36]:
4.7/1878

0.0025026624068157617

In [94]:
criterion = nn.L1Loss()
a=criterion(output, img); 
a

tensor(0.0931, device='cuda:0', grad_fn=<L1LossBackward>)

In [136]:
b=output.padding(2)

AttributeError: 'Tensor' object has no attribute 'padding'

In [145]:
criterion = nn.L1Loss(reduction='none')
a=criterion(output, img)/(img.sum(dim=3).sum(dim=2).sum(dim=1)); 
a.shape
a.mean()   # should i use a.mean * batch size as loss here? 

tensor(0.0046, device='cuda:0', grad_fn=<MeanBackward0>)

In [169]:
a.shape

torch.Size([64, 1, 64, 64])

In [170]:
o=criterion(output, img);
o.shape


torch.Size([64, 1, 64, 64])

In [171]:
s=(img.sum(dim=3).sum(dim=2).sum(dim=1));
s.shape

torch.Size([64])

In [173]:
(o/s).shape

torch.Size([64, 1, 64, 64])

In [176]:
(o/s)[0].mean()

tensor(0.0046, device='cuda:0', grad_fn=<MeanBackward0>)

In [177]:
(o[0]/s[0]).mean()

tensor(0.0032, device='cuda:0', grad_fn=<MeanBackward0>)

In [192]:
(o.view(64,64*64)).sum(dim=1)


tensor([376.2002, 375.2029, 388.9969, 375.9528, 390.7440, 384.9583, 374.2820,
        392.0765, 372.9592, 369.8050, 371.1211, 370.0914, 372.2965, 371.2819,
        373.7209, 378.2418, 387.1617, 385.0990, 374.8397, 393.6779, 379.8616,
        383.6038, 388.1667, 367.0530, 375.0070, 371.9666, 387.4290, 359.9841,
        393.1495, 375.6053, 397.4240, 371.6236, 392.8053, 380.7062, 385.8667,
        375.7635, 371.6948, 391.2544, 369.6983, 375.2678, 392.8057, 376.5280,
        357.5658, 381.0408, 372.9804, 393.4394, 400.0808, 392.5591, 389.8239,
        366.2178, 384.4406, 402.1572, 396.6655, 375.2169, 382.4720, 387.4206,
        373.9464, 378.7662, 389.1381, 382.7741, 369.0723, 389.4471, 396.3343,
        390.2250], device='cuda:0', grad_fn=<SumBackward1>)

In [186]:
s.size()

torch.Size([64])

In [194]:
img[0].sum()

tensor(28.8642, device='cuda:0')

In [181]:
o.size()

torch.Size([64, 1, 64, 64])

In [162]:
(o[0]/s[0]).min()

tensor(0.0004, device='cuda:0', grad_fn=<MinBackward1>)

In [163]:
(o[0]/s[0]).mean()

tensor(0.0032, device='cuda:0', grad_fn=<MeanBackward0>)

In [168]:
criterion = nn.L1Loss(reduction='none')
a=criterion(output, img)/(img.sum(dim=3).sum(dim=2).sum(dim=1)); 
a[0].mean()

tensor(0.0046, device='cuda:0', grad_fn=<MeanBackward0>)

In [159]:
o

tensor([[[[0.0984, 0.0982, 0.0982,  ..., 0.0982, 0.0986, 0.0987],
          [0.0980, 0.0978, 0.0981,  ..., 0.0986, 0.0988, 0.0983],
          [0.0981, 0.0978, 0.0978,  ..., 0.0987, 0.0983, 0.0983],
          ...,
          [0.0988, 0.0984, 0.0981,  ..., 0.0978, 0.0983, 0.0981],
          [0.0987, 0.0980, 0.0986,  ..., 0.0981, 0.0978, 0.0983],
          [0.0984, 0.0988, 0.0981,  ..., 0.0984, 0.0982, 0.0983]]],


        [[[0.0984, 0.0984, 0.0986,  ..., 0.0980, 0.0983, 0.0981],
          [0.0988, 0.0985, 0.0986,  ..., 0.0982, 0.0980, 0.0984],
          [0.0986, 0.0986, 0.0985,  ..., 0.0980, 0.0977, 0.0981],
          ...,
          [0.0979, 0.0982, 0.0984,  ..., 0.0986, 0.0988, 0.0985],
          [0.0980, 0.0981, 0.0980,  ..., 0.0986, 0.0987, 0.0985],
          [0.0981, 0.0982, 0.0983,  ..., 0.0984, 0.0986, 0.0988]]],


        [[[0.0986, 0.0988, 0.0988,  ..., 0.0984, 0.0982, 0.0988],
          [0.0987, 0.0988, 0.0985,  ..., 0.0986, 0.0986, 0.0988],
          [0.0988, 0.0986, 0.0987,  ..

In [147]:
a.sum(dim=3).sum(dim=2).sum(dim=1)

tensor([18.7978, 18.7570, 19.4015, 18.7812, 19.4405, 19.2229, 18.6972, 19.5513,
        18.6488, 18.4426, 18.6093, 18.4840, 18.6372, 18.5100, 18.6942, 18.9117,
        19.2900, 19.2327, 18.7771, 19.6164, 19.0020, 19.1472, 19.3563, 18.3912,
        18.7592, 18.5940, 19.3236, 18.1046, 19.5853, 18.7956, 19.7727, 18.5932,
        19.5860, 19.0441, 19.2512, 18.8355, 18.5655, 19.5026, 18.5566, 18.7135,
        19.5586, 18.7788, 17.9620, 19.0520, 18.6616, 19.5754, 19.9027, 19.5752,
        19.4300, 18.3311, 19.2050, 19.9936, 19.7502, 18.6805, 19.1010, 19.3415,
        18.7298, 18.9430, 19.4094, 19.1276, 18.4948, 19.4263, 19.7307, 19.4690],
       device='cuda:0', grad_fn=<SumBackward1>)

In [133]:
criterion = nn.L1Loss(reduction='mean')

b=torch.zeros(1)
for i in range(64): 
    a=criterion(output[i]/img[i].sum(), img[i]/img[i].sum())
    b=b+a 
    
b    # b is not equal to a 

tensor([0.3011], grad_fn=<AddBackward0>)

In [139]:
criterion = nn.L1Loss(reduction='mean')
criterion(output, img)


tensor(0.0931, device='cuda:0', grad_fn=<L1LossBackward>)

In [134]:
b/64   # b/64 =a 

tensor([0.0047], grad_fn=<DivBackward0>)

In [123]:
criterion(output[0]/img[0].sum(), img[0]/img[0].sum())

tensor(0.0032, device='cuda:0', grad_fn=<L1LossBackward>)

In [119]:
summ=(img.sum(dim=3).sum(dim=2).sum(dim=1));
summ.mean()

tensor(30.4840, device='cuda:0')

In [98]:
img.shape

torch.Size([64, 1, 64, 64])

In [102]:
img.sum().shape

torch.Size([])

In [103]:
img.sum(dim=3).sum(dim=2).sum(dim=1)  # img 64*1*64*64 

tensor([28.8642, 30.7707, 19.7056, 31.3594, 14.3204, 43.1939, 31.3292, 12.9879,
        32.5056, 35.2594, 34.2050, 37.6815, 43.6092, 33.7825, 31.6073, 28.0710,
        17.9027, 19.9654, 33.7577, 11.3865, 34.9876, 57.9737, 16.8976, 40.3368,
        42.4428, 34.4351, 18.1501, 56.5141, 11.9148, 41.8951,  7.6404, 57.7503,
        48.9010, 45.3486, 19.8228, 34.8662, 54.7007, 13.8100, 44.1727, 29.8012,
        12.2587, 28.6363, 48.9836, 33.4935, 62.1392, 11.6250,  4.9836, 12.5665,
        15.2405, 41.1812, 43.9570,  2.9072,  8.3989, 29.8474, 24.8248, 52.9335,
        52.2733, 26.3565, 15.9263, 40.8487, 37.3176, 33.3465,  8.7301, 17.5703],
       device='cuda:0')

In [110]:
img[1].sum()

tensor(30.7707, device='cuda:0')

In [78]:
criterion = nn.L1Loss()
criterion(output, img)

tensor(0.0931, device='cuda:0', grad_fn=<L1LossBackward>)

In [38]:
output.shape

torch.Size([64, 1, 64, 64])

In [70]:
img[0].sum()

tensor(47.3108, device='cuda:0')

In [68]:
img.sum()


tensor(2064.2173, device='cuda:0')

In [72]:
 criterion(output, img)/2064

tensor(3.4420e-06, device='cuda:0', grad_fn=<DivBackward0>)

tensor([0.3011], grad_fn=<AddBackward0>)

In [91]:
b/64 * 1e3

tensor([4.7044], grad_fn=<MulBackward0>)

In [65]:
b/64

tensor([0.0002], grad_fn=<DivBackward0>)

In [25]:
b/64

tensor([0.3468], grad_fn=<DivBackward0>)

In [105]:
b=torch.zeros(1)
for i in range(64): 
    a=criterion(output[i], img[i])
    b=b+a 

RuntimeError: expected device cpu and dtype Float but got device cuda:0 and dtype Float

In [44]:
b/img.sum()

tensor([0.1611], grad_fn=<DivBackward0>)

In [31]:
img[0].sum()

tensor(4.0610, device='cuda:0')

In [66]:
img.sum()

tensor(2064.2173, device='cuda:0')

In [71]:
criterion(output/img.sum(), img/img.sum())

tensor(3.4416e-06, device='cuda:0', grad_fn=<L1LossBackward>)

In [47]:
criterion(z[:,:7]

SyntaxError: unexpected EOF while parsing (<ipython-input-47-f29687896808>, line 1)

In [50]:
 criterion(output, img)

tensor(20.8707, device='cuda:0', grad_fn=<L1LossBackward>)

In [55]:
print('epoch [{}/{}], loss:{:.6f}, MSE_loss:{:.6f}, recon_loss:{:.6f}, latent_loss:{:.6f}'
          .format(epoch + 1, num_epochs, total_loss / num_examples, total_mse/ num_examples, total_recon / num_examples, total_latent/ num_examples))   

epoch [5/20000], loss:0.063595, MSE_loss:0.006816, recon_loss:0.063595, latent_loss:0.006816


In [203]:
        loss= ( criterion_none(output, img)/(img.sum(dim=3).sum(dim=2).sum(dim=1))) #  .mean())  #* batch_size # + criterion_mean(z[:,:7], label) /(1e9) 


RuntimeError: The size of tensor a (64) must match the size of tensor b (16) at non-singleton dimension 3

In [204]:
criterion_none(output, img).shape

torch.Size([16, 1, 64, 64])